# Visualizing emission datasets in Python

The goal of this Notebook is to download VOC emission datasets online, and compare those by plotting them. We will focus on datasets from the EU SEEDS project (https://www.seedsproject.eu/) and on datasets available at the Athmosphere Data Store (https://ads.atmosphere.copernicus.eu/). 

## Initial setup

To start, we will install and import some libraries that we will need throughout this Notebook.

In [ ]:
!pip install cartopy xarray[complete] zarr==2.18.3 numcodecs==0.15.1 cdsapi

In [ ]:
# Core scientific stack
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt

# Mapping
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Utilities
from mpl_toolkits.axes_grid1 import make_axes_locatable
import fsspec  # for cloud data access
import netCDF4  # netCDF support

# increases the standard font size in figures
plt.rcParams.update({'font.size': 18})

## Download SEEDS data

Next, we will download SEEDS datasets from the https://seedsproject.eu website. The data portal provides some information on how to download the data in Python. The data format is .zarr, hence we need to use the xarray library which includes a wrapper for .zarr files. Using the url provided on the website, we can access the top-down biomass burning emission dataset of SEEDS.

In [ ]:
# Open remote dataset
url = "https://data.seedsproject.eu/seeds_biomass-burning-carbon-flux_bira-iasb_20180101-20221231_magritte_v2/slices.zarr"
store = fsspec.get_mapper(url)
ds = xr.open_zarr(store = store)
ds

The dataset contains latitude, longitude, and time coordinates, as well as the isoprene flux. We can store specific chunks of data in netCDF format to store locally.

In [ ]:
CONFIG = {
    "data_dir": "",
    "time_range": slice("2021-01-01", "2021-12-31"),
    "lat_bounds": slice(35, 42),
    "lon_bounds": slice(20, 27),
}

In [ ]:
# Subset a time range, save as NetCDF
time_subset = ds.pyrogenic_totalc_flux.sel(time=CONFIG["time_range"])
time_subset.to_netcdf(CONFIG["data_dir"]+'pyrogenic_totalc_flux_time_subset.nc')

# Subset a time and geographical range, save as NetCDF
time_geo_subset = ds.pyrogenic_totalc_flux.sel(time=CONFIG["time_range"], latitude=CONFIG["lat_bounds"], longitude=CONFIG["lon_bounds"])
time_geo_subset.to_netcdf(CONFIG["data_dir"]+'pyrogenic_totalc_flux_time_geo_subset.nc')



Let's open the dataset and look at its contents.

In [ ]:
nc = netCDF4.Dataset(CONFIG["data_dir"]+'pyrogenic_totalc_flux_time_geo_subset.nc','r')
nc

In [ ]:
nc = netCDF4.Dataset(CONFIG["data_dir"]+'pyrogenic_totalc_flux_time_subset.nc','r')
nc.variables['pyrogenic_totalc_flux']

## Plot SEEDS data

The dataset contains the total carbon flux over Europe due to biomass burning events. Let's plot the total emissions using cartopy.

In [ ]:
lat_seeds = nc.variables['latitude'][:]
lon_seeds = nc.variables['longitude'][:]
time = nc.variables['time'][:]
flux_seeds = nc.variables['pyrogenic_totalc_flux'][:,:,:]

Define the plot routine

In [ ]:
def plotmap_cartopy(lonplot, latplot, borders, plotdata, label, colormap, vmin, vmax, steps, lognorm):
    # This function plots a dataset with longitude and latitude coordinates
    # onto a cartopy map. It takes arguments for the figure label, colormap,
    # minimum and maximum values of the colorbar, and whether to plot the colorbar
    # logarithmically or not.
    
    fig, ax = plt.subplots(1, 1, figsize=(10,10),
                    subplot_kw={'projection': ccrs.PlateCarree()})

    ax.set_extent(borders, crs=ccrs.PlateCarree())
    ax.coastlines()
    gls = ax.gridlines(draw_labels = True)
    gls.top_labels = False
    gls.right_labels = False
    ax.add_feature(cfeature.BORDERS)

    # define colormap
    cmap = plt.get_cmap(colormap)
    cmap = cmap(np.linspace(0,1,steps))
    cmap = mpl.colors.ListedColormap(cmap)

    # plot the data
    lonplotmap, latplotmap = np.meshgrid(lonplot, latplot)
    if lognorm:
        h = ax.pcolormesh(lonplotmap,latplotmap,plotdata,cmap = cmap, transform=ccrs.PlateCarree(), norm=mpl.colors.LogNorm(vmin=vmin, vmax=vmax))
    else:
        h = ax.pcolormesh(lonplotmap,latplotmap,plotdata,cmap = cmap, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)

    # plot the colorbar
    divider = make_axes_locatable(ax)
    ax_cb = divider.new_horizontal(size="3%", pad=0.1, axes_class=plt.Axes)
    cbar = plt.colorbar(h, label=label, cax=ax_cb, orientation ='vertical')
    
    fig.add_axes(ax_cb)

In [ ]:
# take the average over the time dimension of the total C data 
flux_seeds_av = np.nanmean(flux_seeds[:,:,:], axis = 0)

vmin = np.nanmax(flux_seeds_av)*0.0001 # lower edge of colorbar
vmax = np.nanmax(flux_seeds_av) # Upper edge of colorbar
steps = 20 # number of steps in colorbar
lognorm = True # logarithmic colorbar
colormap = 'YlOrRd' # colorbar
borders = [lon_seeds[0], lon_seeds[-1], lat_seeds[0], lat_seeds[-1]] # edges of the plotdomain
figlabel = 'Biomass burning total C emissions \n kg C m-2 s-1'

# this function plots the map
plotmap_cartopy(lon_seeds, lat_seeds, borders, flux_seeds_av, figlabel, colormap, vmin, vmax, steps, lognorm)

## Download CAMS data

As a next step, it would be interesting to see how this compares to bottom emissions inventories. There are many different options available, e.g. GFED4s, QFEDv2.4, Finn, GFAS, ... In this workshop, let's compare with the GFAS inventory of CAMS. It can be accessed through the Atmosphere Data Store (ADS). 

To set up your ADS API credentials, please login/register on the ADS, then you will see your unique [API key here](https://ads.atmosphere.copernicus.eu/how-to-api).
You can add this API key to your current session by replacing ######### in the code cell below with your API key.


In [ ]:
import os
os.environ['CDSAPI_URL'] = 'https://ads.atmosphere.copernicus.eu/api'
os.environ['CDSAPI_KEY'] = '###########################################'

Visit the download form for the [GFAS global fire emissions data](https://ads.atmosphere.copernicus.eu/datasets/cams-global-fire-emissions-gfas?tab=overview). View the parameters in the API script in the following cell and select the corresponding options. For each year there will be a netcdf file inside the zip that will be downloaded. The data is available from year 2000 up to 2020.

At the end of the download form, select "Show API request". This will reveal a block of code, which should be identical to the code cell below.

In [ ]:
variable = 'wildfires_overall_flux_of_burnt_carbon'
year = '2021'

In [ ]:
import cdsapi

dataset = "cams-global-fire-emissions-gfas"
request = {
    "variable": ["wildfire_overall_flux_of_burnt_carbon"],
    "date": ["2021-01-01/2021-12-31"],
    "data_format": "netcdf"
}

client = cdsapi.Client()
client.retrieve(dataset, request).download(f'{CONFIG['data_dir']}{dataset}.nc')

Now, the netCDF dataset has been downloaded to your target folder. Lets check its contents.

In [ ]:
nc = netCDF4.Dataset('cams-global-fire-emissions-gfas.nc','r')
nc

The dataset also has daily emissions at 0.1x0.1 degrees resolution, similar to SEEDS. However, GFAS is a global emission inventory, hence has a much larger domain.

In [ ]:
lat_gfas = nc.variables['latitude'][:]
lon_gfas = nc.variables['longitude'][:]
flux_gfas = nc.variables['cfire'][:,:,:]

## Plot GFAS data

Plot this dataset in the same way as we plotted the SEEDS dataset before.

In [ ]:
# take the average over the time dimension of the total C data 
flux_gfas_av = np.nanmean(flux_gfas, axis = 0)

plotmap_cartopy(lon_gfas, lat_gfas, borders, flux_gfas_av, figlabel, colormap, vmin, vmax, steps, lognorm)

It is clear from a visual expection that the fire emissions in the two inventories are in agreement in terms of fire locations. The SEEDS inventory appears to have higher agriculatural waste burning emissions than GFAS in areas like Ukraine and Russia, or the border between Romania and Bulgaria. Let's look at a specific fire event of your choice. Change the borders of the domain and the time coordinate to match that of the forest fire.

In [ ]:
# e.g. Greece borders
borders = [20, 27, 35, 42]

# e.g. the month of August
nday_mon = np.array([31,28,31,30,31,30,31,31,30,31,30,31])
iday0 = np.sum(nday_mon[:7])
iday1 = iday0 + 31

flux_seeds_av = np.nanmean(flux_seeds[iday0:iday1,:,:], axis = 0)
flux_gfas_av = np.nanmean(flux_gfas[iday0:iday1,:,:], axis = 0)

plotmap_cartopy(lon_seeds, lat_seeds, borders, flux_seeds_av, figlabel, colormap, vmin, vmax, steps, lognorm)
plotmap_cartopy(lon_gfas, lat_gfas, borders, flux_gfas_av, figlabel, colormap, vmin, vmax, steps, lognorm)

The locations of the fires agree very well, which is mainly because GFAS uses the MODIS radiative power dataset, whereas the top-down emissions rely on QFED as a priori information, which also uses MODIS radiative fire. However, in order to get a more quantitative comparison of the two inventories, let's make a time series analysis of the fires and calculate the total carbon emitted.

## Time series comparison

In [ ]:
###### GFAS ######
# indices of lats and lons of borders
ilats = np.argwhere((borders[2]<=lat_gfas)&(borders[3]>lat_gfas)).flatten()
ilons = np.argwhere((borders[0]<=lon_gfas)&(borders[1]>lon_gfas)).flatten()


# It is more interesting to derive the monthly total emissions in units of Tg per month over the domain. 
# To do this, we need to multiply each gridcell with its surface area and with seconds per month.
# flux_gfas is in kg m-2 s-1, so use the appropriate units.
Rearth2 = 6.371e6**2 # m2
deg_to_rad = 3.14159/180.
nday_mon = np.array([31,28,31,30,31,30,31,31,30,31,30,31])
secday = 3600*24
constant = 1e-9 # to go from kg to Tg

surface_area_gfas = np.empty((len(lat_gfas), len(lon_gfas)))
# get the resolution of the grid in delta_lon, delta_lat
delta_lon = np.abs(lon_gfas[1]-lon_gfas[0])
delta_lat = np.abs(lat_gfas[1]-lat_gfas[0])
for ilat in range(len(lat_gfas)):
    area = Rearth2*(np.sin((lat_gfas[ilat]+delta_lat*0.5)*deg_to_rad)-np.sin((lat_gfas[ilat]-delta_lat*0.5)*deg_to_rad))*delta_lon*deg_to_rad
    for ilon in range(len(lon_gfas)):
        surface_area_gfas[ilat,ilon] = area


# now we can calculate the total fluxes
# take just one month of data
flux_gfas_mon = flux_gfas[iday0:iday1,:,:]
total_flux_gfas = secday * constant * np.nansum(flux_gfas_mon[np.ix_(np.arange(flux_gfas_mon.shape[0]),ilats,ilons)]*surface_area_gfas[np.ix_(ilats,ilons)],axis = (1,2))

###### SEEDS #####
# do the same for the SEEDS coordinate system
# flux_seeds is in the same units as GFAS
ilats = np.argwhere((borders[2]<=lat_seeds)&(borders[3]>lat_seeds)).flatten()
ilons = np.argwhere((borders[0]<=lon_seeds)&(borders[1]>lon_seeds)).flatten()

surface_area_seeds = np.empty((len(lat_seeds), len(lon_seeds)))
delta_lon = np.abs(lon_seeds[1]-lon_seeds[0])
delta_lat = np.abs(lat_seeds[1]-lat_seeds[0])
for ilat in range(len(lat_seeds)):
    area = Rearth2*(np.sin((lat_seeds[ilat]+delta_lat*0.5)*deg_to_rad)-np.sin((lat_seeds[ilat]-delta_lat*0.5)*deg_to_rad))*delta_lon*deg_to_rad
    for ilon in range(len(lon_seeds)):
        surface_area_seeds[ilat,ilon] = area


# now we can calculate the total fluxes
# take just one month of data
flux_seeds_mon = flux_seeds[iday0:iday1,:,:]

total_flux_seeds = secday * constant * np.nansum(flux_seeds_mon[np.ix_(np.arange(flux_seeds_mon.shape[0]),ilats,ilons)]*surface_area_seeds[np.ix_(ilats,ilons)],axis = (1,2))


Make the plot routine for time series

In [ ]:
def plot_timeseries(xvals, data, datalabels, colorlist, xlabel, ylabel, xticklabels, figname): 
    # This function allow to plot different datasets onto a time series plot.
    # The xvals determines the x-coordinates of the figure, the xticklabels (if any) can be used
    # to substitute the x scale with the month names.

    
    fig, ax = plt.subplots(1,1,figsize =(len(xvals),8))

    Ndatasets = len(datalabels) # number of datasets to plot

    for n in range(Ndatasets):
        if n == 0:
            # first dataset
            lns = ax.plot(xvals, data[n,:], color = colorlist[n], marker = 'o', markersize = 16, linestyle ='-', lw = 4, label = datalabels[n])
        else:
            # other datasets
            ln = ax.plot(xvals, data[n,:], color = colorlist[n], marker = 'o', markersize = 16, linestyle ='-', lw = 4, label = datalabels[n])
            lns = lns + ln

    trans = mpl.transforms.blended_transform_factory(ax.transData, ax.transAxes)

    ax.set_xlim(-0.8, xvals[-1]+0.8)
    ax.set_ylim(0.,1.1*(np.nanmax(data)))

    # make xticklabels for the plot, for example to put months or other dates
    if xticklabels is not None:
        ax.set_xticks(ticks = xvals, labels = xticklabels)
    
    ax.tick_params(axis = 'x', which='major', direction = 'in', length = 5, width = 2, top = True)
    ax.tick_params(axis = 'y', which='major', direction = 'in', length = 5, width = 2, right = True)

    # plot gridlines to separate the months
    minor_locator = mpl.ticker.AutoMinorLocator(2)
    ax.xaxis.set_minor_locator(minor_locator)
    ax.grid(which='minor')
    
    ax.set_ylabel(ylabel)

    labs = [l.get_label() for l in lns]
    ax.legend(lns, labs, loc=1,fontsize=24)
    
    # fig.savefig(figname,dpi=300,format='png',bbox_inches='tight') # to save the figure as png in high resolution
    plt.show()
    plt.close()


In [ ]:
# definitions for the time series plot
xvals = np.arange(total_flux_gfas.shape[0])
data = np.array([total_flux_gfas, total_flux_seeds])
datalabels = ['GFAS', 'SEEDS top-down']
colorlist = ['C0', 'C1']
xlabel = 'Days'
ylabel = 'Total carbon emissions (Tg C)'
# xticklabels = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']
xticklabels=None
figname = 'figure_name.png'


plot_timeseries(xvals, data, datalabels, colorlist, xlabel, ylabel, xticklabels, figname)